In [84]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
import sys
import importlib
sys.path.append('/kaggle/mtn_proj_ticks/')
import climbing_ticks_helper as helper
importlib.reload(helper)
pd.set_option('display.expand_frame_repr', False) # display full data in terminal
%matplotlib inline
bar = 'bar'

In [106]:

df = pd.read_csv('/kaggle/mtn_proj_ticks/model_ready_ticks.csv')

# first take at a Random Forest model:
X = df.drop(columns=['Attempts', 'Lead Style'])
y = df[['Attempts', 'Lead Style']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

# compute accuracy, recall, precision, f1 score for multi class predictions
# Generate classification reports for each response variable
for resp in y_test.columns:
    print(f"Classification report for response variable {resp}:")
    # don't print macro avg or weighted avg
    cr = classification_report(y_test[resp], y_pred_df[resp], zero_division=0, output_dict=True)
    cr.pop('macro avg', None)
    cr.pop('weighted avg', None)
    prettytable = pd.DataFrame(cr).T
    print(prettytable.round(2))
    print("\n")


Classification report for response variable Attempts:
          precision  recall  f1-score  support
1              0.87    0.97      0.91   176.00
2              0.10    0.04      0.06    26.00
3              0.00    0.00      0.00     1.00
4              0.00    0.00      0.00     3.00
accuracy       0.83    0.83      0.83     0.83


Classification report for response variable Lead Style:
          precision  recall  f1-score  support
0              0.85    0.95      0.90   149.00
1              0.40    0.32      0.36    25.00
2              0.37    0.22      0.27    32.00
accuracy       0.76    0.76      0.76     0.76




In [112]:
df_readable = pd.read_csv('/kaggle/mtn_proj_ticks/grouped_ticks.csv')
df_readable = df_readable[df_readable['RouteID'].isin(X_test['RouteID'])].set_index('RouteID').loc[X_test['RouteID']].reset_index()
df_combined = helper.combine_predictions_with_data(df_readable, y_pred_df)

# print all rows that got Lead Style wrong
wrong_preds = df_combined[df_combined['Lead Style'] != df_combined['Predicted Lead Style']]

In [ ]:
# TODO:
# * categorize predictions that are close to the correct answer. EG: predicting an Onsight/Flash when the actual is a Redpoint with only 2 attempts isn't that bad. but predicting an Onsight/Flash when the actual is a Fell/Hung with 5 attempts is bad.
# * categorize predictions that though I woudl do better, vs predictions that thought I would do worse